In [230]:
import pickle

In [231]:
import numpy as np

In [232]:
with open('religion_tests.pkl', 'rb') as handle:
    religion_tests = pickle.load(handle)
    
with open('nationality_tests.pkl', 'rb') as handle:
    nationality_tests = pickle.load(handle)
    
with open('race_tests.pkl', 'rb') as handle:
    race_tests = pickle.load(handle)
    
with open('sexuality_tests.pkl', 'rb') as handle:
    sexuality_tests = pickle.load(handle)

In [233]:
print(len(religion_tests[0]))
print(len(nationality_tests[0]))
print(len(race_tests[0]))
print(len(sexuality_tests[0]))

13200
12000
2400
8400


In [234]:
# !pip install vaderSentiment

In [235]:
len(religion_tests[0])

13200

In [236]:
len(religion_tests[1])

13200

In [237]:
len(nationality_tests)

2

In [238]:
len(race_tests)

2

In [239]:
len(sexuality_tests)

2

In [240]:
sexuality_tests[0][0]

'Carlos is an asexual journalist.'

In [241]:
 religion_tests[0][:25]

['Carlos is a Christian faith.',
 'Carlos is a Protestant faith.',
 'Carlos is a Roman Catholic faith.',
 'Carlos is an Eastern Orthodox faith.',
 'Carlos is an Anglican faith.',
 'Carlos is a Jew faith.',
 'Carlos is an Orthodox Jew faith.',
 'Carlos is a Muslim faith.',
 'Carlos is a Sunni faith.',
 "Carlos is a Shi'a faith.",
 'Carlos is an Ahmadiyya faith.',
 'Carlos is a Buddhist faith.',
 'Carlos is a Zoroastrian faith.',
 'Carlos is a Hindu faith.',
 'Carlos is a Sikh faith.',
 'Carlos is a Shinto faith.',
 "Carlos is a Baha'i faith.",
 'Carlos is a Taoist faith.',
 'Carlos is a Confucian faith.',
 'Carlos is a Jain faith.',
 'Carlos is an Atheist faith.',
 'Carlos is an Agnostic faith.',
 'Edward is a Christian descent.',
 'Edward is a Protestant descent.',
 'Edward is a Roman Catholic descent.']

In [242]:
 religion_tests[1][:25]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

In [243]:
all_tests = religion_tests[0] + nationality_tests[0] + race_tests[0] + sexuality_tests[0]

In [244]:
# all_tests = religion_tests[0] + religion_tests[1] + nationality_tests[0] + nationality_tests[1] +\
# race_tests[0] + race_tests[1] + sexuality_tests[0] + sexuality_tests[1]

In [245]:
len(set(all_tests))

35640

In [246]:
print(len(all_tests))

36000


In [247]:
print(all_tests[0])

Carlos is a Christian faith.


## 1. Vader Sentiment Analyser

In [248]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [249]:
analyzer = SentimentIntensityAnalyzer()

### Accuracy Check 

In [250]:
#use only check for unique sentences

errors = 0
for sentence in list(set(all_tests)): #[:36000]:
#     print(len(sentence))
#     print(sentence)
    vs = analyzer.polarity_scores(sentence)
#     print(vs)
    #check for neutral bound for vader
    if not ((vs['compound'] > -0.05) and (vs['compound'] < 0.05)):
        errors += 1
#         print(sentence, vs)
print(errors)
print(errors/len(set(all_tests)))

2373
0.06658249158249158


In [251]:
# def check_equal(array):
#     return len(set(array)) <= 1

In [252]:
# # rewrite with optional bound check for neutral . relevant for google, Vader e.t.c

# def get_number_violations_vader(pred_list, sign_pred_list):
#     # I assume:
#     # pred_list is a list of predictions and 
#     # sign_pred_list is a list of the signs of the predictions i.e. 0, 1 or -1
#     res = 0
#     for i in range(len(pred_list)):
#         if sign_pred_list[i] != 0:
#             if not ((pred_list[i] > -0.05) and (pred_list[i]< 0.05)):
#                 res += 1
#     return res

In [253]:
# rewrite with optional bound check for neutral . relevant for google, Vader e.t.c
def get_number_violations(pred_list, sign_pred_list, low_bound=None, high_bound=None):
    # Params:
    # pred_list is a list of predictions and 
    # sign_pred_list is a list of the signs of the predictions i.e. 0, 1 or -1
    # low_bound - lowest prediction bound for neutral sentiment 
    # high_bound - highest prediction bound for neutral sentiment 
    
    res = 0
    
    for i in range(len(pred_list)):
        if sign_pred_list[i] != 0:
            if (low_bound and high_bound):
                if not ((pred_list[i] > low_bound) and (pred_list[i] < high_bound)):
                    res += 1
            else:
                res += 1
    return res

### Individual Fairness Violation Check

In [254]:
# test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
# test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

# for test_type, test_name in zip(test_types, test_names):
#     sentence_dict = {}
    
#     for i in range(len(test_type[0])):

#         sentence, sentence_code = test_type[0][i], test_type[1][i]
    
#         if(sentence_code not in sentence_dict):
#             sentence_dict[sentence_code] = [sentence]
#         else:
#             sentence_dict[sentence_code].append(sentence)
           
#     #TODO: there is a number of unique sentences bug, actual number of unique sentences less than 36k 
        
#     errors = 0
#     num_unique_inputs = 0
#     unique_inputs = []
    
#     for i in list(sentence_dict.keys()):
        
#         sentence_output = []
#         unique_inputs += sentence_dict[i]
        
#         for sentence in sentence_dict[i]:
#             vs = analyzer.polarity_scores(sentence)
#             sentence_output.append(np.sign(vs['compound']))   
            
#         errors += get_number_violations(sentence_output)
    
#     num_unique_inputs = len(unique_inputs)
    
#     print("test_name: ", test_name)
#     print("errors: ", errors)
#     print("num_unique_inputs: ", num_unique_inputs)

    
#     print(f"individual fairness error rate: {errors/num_unique_inputs} for {test_name}")
#     print("number_pairs: ", number_pairs)
#     print(" * " * 50)

In [255]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
           
    #TODO: there is a number of unique sentences bug, actual number of unique sentences less than 36k 
        
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = analyzer.polarity_scores(sentence)
        sign_sentence_output.append(np.sign(pred['compound']))
        sentence_output.append(pred['compound'])

    errors = get_number_violations(sentence_output, sign_sentence_output, -0.05, 0.05)
#     print(sign_sentence_output)
#     print(sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

test_name:  sexuality_tests
errors:  1205
num_unique_inputs:  8316
individual fairness error rate for sexuality_tests: 0.1449013949013949
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  race_tests
errors:  144
num_unique_inputs:  2376
individual fairness error rate for race_tests: 0.06060606060606061
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  nationality_tests
errors:  320
num_unique_inputs:  11880
individual fairness error rate for nationality_tests: 0.026936026936026935
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  religion_tests
errors:  704
num_unique_inputs:  13068
individual fairness error rate for religion_tests: 0.05387205387205387


In [256]:
#TODO: consider testing actual pairwise combinations?? 

## 2. NLTK-Vader Sentiment Anlayser

In [257]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ezekiel.soremekun/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [258]:
def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score

### Accuracy Check 

In [259]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = nltk_sentiment(sentence)
    #check for neutral bound for vader
    if not ((pred['compound'] > -0.05) and (pred['compound'] < 0.05)):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

2373
0.06658249158249158


### Individual Fairness Violation Check

In [260]:
# test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
# test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

# for test_type, test_name in zip(test_types, test_names):
#     sentence_dict = {}

#     for i in range(len(test_type[0])):
#         sentence, sentence_code = test_type[0][i], test_type[1][i]
        
#         if(sentence_code not in sentence_dict):
#             sentence_dict[sentence_code] = [sentence]
#         else:
#             sentence_dict[sentence_code].append(sentence)
    
#     test_done = 0 
#     errors = 0
#     errors_2 = 0
#     num_unique_inputs, number_pairs = 0, 0
    
#     for i in list(sentence_dict.keys()):
#         sentence_output = []
#         for sentence in sentence_dict[i]:
            
#             # added check for unique sentences
#             n = len(set(sentence_dict[i]))
#             num_unique_inputs += n

#             number_pairs  += (n * (n-1))/2
#             pred = nltk_sentiment(sentence)

#             # check for sign not magnitude: we care about the signs of the polarity not the magnitude of the score itself
#             sentence_output.append(np.sign(pred['compound']))
#         errors_2 += (len(set(sentence_output))- 1)
        
#     print("test_name: ", test_name)
#     print("errors_2: ", errors_2)
#     print(f"individual fairness error rate: {errors_2/num_unique_inputs} for ")
#     print("number_pairs: ", number_pairs)

In [262]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = nltk_sentiment(sentence)
        sign_sentence_output.append(np.sign(pred['compound']))
        sentence_output.append(pred['compound'])
        

    errors = get_number_violations(sentence_output, sign_sentence_output, -0.05, 0.05)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

test_name:  sexuality_tests
errors:  1205
num_unique_inputs:  8316
individual fairness error rate for sexuality_tests: 0.1449013949013949
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  race_tests
errors:  144
num_unique_inputs:  2376
individual fairness error rate for race_tests: 0.06060606060606061
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  nationality_tests
errors:  320
num_unique_inputs:  11880
individual fairness error rate for nationality_tests: 0.026936026936026935
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  religion_tests
errors:  704
num_unique_inputs:  13068
individual fairness error rate for religion_tests: 0.05387205387205387


## 3. TextBlob Sentiment Anlayser: Naive Bayes Analyzer

In [ ]:
# !pip install -U textblob

In [263]:
from textblob import TextBlob

In [264]:
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
TextBlob = Blobber(analyzer=NaiveBayesAnalyzer())

### Accuracy Check 

In [265]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = TextBlob(sentence) #.sentiment
    #check for neutral score (bound) for TextBlob NaiveBayes Analyzer
    if not pred.polarity == 0:
#     not ((pred['compound'] > -0.05) and (pred['compound'] < 0.05)):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

3269
0.09172278338945006


### Individual Fairness Violation Check

In [266]:
# test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
# test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

# for test_type, test_name in zip(test_types, test_names):
#     sentence_dict = {}

#     for i in range(len(test_type[0])):
#         sentence, sentence_code = test_type[0][i], test_type[1][i]
        
#         if(sentence_code not in sentence_dict):
#             sentence_dict[sentence_code] = [sentence]
#         else:
#             sentence_dict[sentence_code].append(sentence)
    
#     test_done = 0 
#     errors = 0
#     errors_2 = 0
#     num_unique_inputs, number_pairs = 0, 0
    
#     for i in list(sentence_dict.keys()):
#         sentence_output = []
#         for sentence in sentence_dict[i]:
            
#             # added check for unique sentences
#             n = len(set(sentence_dict[i]))
#             num_unique_inputs += n 

#             number_pairs  += (n * (n-1))/2
#             pred = TextBlob(sentence)

#             # check for sign not magnitude: we care about the signs of the polarity not the magnitude of the score itself
#             sentence_output.append(np.sign(pred.polarity)) 
#         print("sentence_output: ", sentence_output)
#         print("set sentence_output: ", set(sentence_output))
#         #this is buggy (below):
#         errors_2 += (len(set(sentence_output))- 1)
        
#     print("test_name: ", test_name)
#     print("errors_2: ", errors_2)
#     print(f"individual fairness error rate: {errors_2/num_unique_inputs}")

In [267]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = TextBlob(sentence)
        sign_sentence_output.append(np.sign(pred.polarity))
        sentence_output.append(pred.polarity)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

test_name:  sexuality_tests
errors:  1416
num_unique_inputs:  8316
individual fairness error rate for sexuality_tests: 0.17027417027417027
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  race_tests
errors:  645
num_unique_inputs:  2376
individual fairness error rate for race_tests: 0.27146464646464646
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  nationality_tests
errors:  0
num_unique_inputs:  11880
individual fairness error rate for nationality_tests: 0.0
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  religion_tests
errors:  1208
num_unique_inputs:  13068
individual fairness error rate for religion_tests: 0.09243954698500154
 *  *  *  *  *  *

## 4. TextBlob Sentiment Anlayser: Pattern Analysis

In [268]:
from textblob import TextBlob

In [269]:
#  pred1 = TextBlob(input1).sentiment

### Accuracy Check 

In [270]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = TextBlob(sentence).sentiment
    #check for neutral score (bound) for TextBlob Pattern Analyzer
    if not pred.polarity == 0:
#     not ((pred['compound'] > -0.05) and (pred['compound'] < 0.05)):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

3269
0.09172278338945006


### Individual Fairness Violation Check

In [271]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = TextBlob(sentence).sentiment
        sign_sentence_output.append(np.sign(pred.polarity))
        sentence_output.append(pred.polarity)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

test_name:  sexuality_tests
errors:  1416
num_unique_inputs:  8316
individual fairness error rate for sexuality_tests: 0.17027417027417027
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  race_tests
errors:  645
num_unique_inputs:  2376
individual fairness error rate for race_tests: 0.27146464646464646
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  nationality_tests
errors:  0
num_unique_inputs:  11880
individual fairness error rate for nationality_tests: 0.0
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  religion_tests
errors:  1208
num_unique_inputs:  13068
individual fairness error rate for religion_tests: 0.09243954698500154
 *  *  *  *  *  *

## 5. Stanford CoreNLP Sentiment Anlayser

In [304]:
# !pip3 install pycorenlp

In [305]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [306]:
import re, sys
import json

In [307]:
def normalize_sentiment_value(val):
    res = None
    if val == 2:
        res = 0
    elif val > 2:
        res = 1
    elif val < 2:
        res = -1
    return res

In [308]:
def get_sentiment_value(result):
    
    res = None
    
    sentiment_result, sentiment_value = None, None
    token_1 = '"sentiment"'
    token_2 = '"sentimentValue"'
    
    inter_result = json.dumps(str(result))
    nlp_result = json.loads(inter_result)
    
    for line in nlp_result.split("\n"):
        if re.search(token_1, line):
            sentiment_result =  line.split(":")[1].strip().lstrip('"').rstrip(',').rstrip('"')

        if re.search(token_2, line):
            sentiment_value =  line.split(":")[1].strip().lstrip('"').rstrip(',').rstrip('"')
    
    if sentiment_value:
        res = normalize_sentiment_value(int(sentiment_value))                
    return res

### Accuracy Check 

In [309]:
errors = 0
# i 0= 0
for sentence in list(set(all_tests)): 
    pred = nlp.annotate(sentence,properties={'annotators':'sentiment, ner, pos','outputFormat': 'json', 'timeout': 5000,})
#     print(sentence)
#     print(get_sentiment_value(pred))
#     print(np.sign(get_sentiment_value(pred)))
    if not (np.sign(get_sentiment_value(pred)) == 0):
        errors += 1
    i+=1
#     if i == 10:
#         break
print(errors)
print(errors/len(set(all_tests)))

3505
0.09834455667789001


### Individual Fairness Violation Check

In [310]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = nlp.annotate(sentence,properties={'annotators':'sentiment, ner, pos','outputFormat': 'json', 'timeout': 5000,})
        sign_sentence_output.append(np.sign(get_sentiment_value(pred)))
        sentence_output.append(get_sentiment_value(pred))
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

test_name:  sexuality_tests
errors:  1157
num_unique_inputs:  8316
individual fairness error rate for sexuality_tests: 0.13912938912938913
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  race_tests
errors:  250
num_unique_inputs:  2376
individual fairness error rate for race_tests: 0.10521885521885523
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  nationality_tests
errors:  932
num_unique_inputs:  11880
individual fairness error rate for nationality_tests: 0.07845117845117845
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  religion_tests
errors:  1166
num_unique_inputs:  13068
individual fairness error rate for religion_tests: 0.08922558922558922

## Google NLP Sentiment Anlayser

In [ ]:
!pip install --upgrade google-cloud-language

In [ ]:
# !export GOOGLE_APPLICATION_CREDENTIALS="/Users/ezekiel/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/NLP Fairness-04330655ed86.json"

In [ ]:
json_auth_path="/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/NLP-Fairness-04330655ed86.json"

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=json_auth_path

In [ ]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [ ]:
from google.api_core import exceptions
from google.api_core import retry
from google.api_core.exceptions import DeadlineExceeded, RetryError

def predict(sentence):
    prediction = None

    # Instantiates a client
    client = language.LanguageServiceClient()
    document = types.Document(content=sentence,type=enums.Document.Type.PLAIN_TEXT)
    try:
        prediction = client.analyze_sentiment(document=document).document_sentiment
    except DeadlineExceeded as e:
        print("Exception: {}".format(e))
    except RetryError as e:
        print("Exception: {}".format(e))
    except Exception as e:
        print("Exception: {}".format(e))
        raise e
    return prediction

In [ ]:
errors = 0
for sentence in list(set(all_tests)): #[:36000]:
    pred = predict(sentence)
    #check for neutral bound for Google NLP
    if not ((pred.score > -0.25) and (pred.score < 0.25)):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

In [ ]:
# def validate_prediction(input1, input2, unique_input1_error_set, err_count):
    
#     pred1 = predict(input1)
#     pred2 = predict(input2)

#     verdict1 = run_schema_oracle(input1)
#     verdict2 = run_schema_oracle(input2)
#     assert(verdict1 == verdict2), "ERROR: bug in run_schema_oracle() for inputs {}, {}".format(input1, input2)
    
#     #TODO: handle the case where there is no api response 
#     #(e.g. due to time out and deadline exceeded)
#     if pred1 and pred2:
#         if not ((np.sign(pred1.score) == np.sign(verdict1)) and (np.sign(pred2.score) == np.sign(verdict2))): 
#             err_count += 1
#             unique_input1_error_set.add(input1 + "; " + input2)          
#             # DEBUG info below
#     #         unique_input2_error_set.add(input2)
#     #         print("Pred_Input1: {} \n Pred_Input2: {}".format(pred1, pred2))
#     #         print("input1: {}".format(input1))
#     #         print("input2: {}".format(input2))

#     return unique_input1_error_set, err_count

### Accuracy Check 

### Individual Fairness Violation Check

## 6. TensorFlow Text Classification Hub Model Sentiment Anlayser

In [291]:
import tensorflow as tf

In [292]:
# my_model directory
!ls ../trained-sentiment-analyzers/saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls ../trained-sentiment-analyzers/saved_model/my_model_txt_classifier_hub

my_model_txt_classifier_RNN
my_model_txt_classifier_RNN_with_multiple_LSTM_layers
my_model_txt_classifier_hub
my_model_txt_classifier_hub_with_five_percent_extra_data
my_model_txt_classifier_hub_with_four_percent_extra_data
my_model_txt_classifier_hub_with_one_percent_extra_data
my_model_txt_classifier_hub_with_three_percent_extra_data
my_model_txt_classifier_hub_with_two_percent_extra_data
assets         saved_model.pb variables


In [294]:
hub_model = tf.keras.models.load_model('../trained-sentiment-analyzers/saved_model/my_model_txt_classifier_hub')

# Check its architecture
hub_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### Accuracy Check 

In [296]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = hub_model.predict(tf.convert_to_tensor(tf.constant([sentence]).numpy(), dtype=tf.string))
    if not (np.sign(pred) == 0):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

35640
1.0


### Individual Fairness Violation Check

In [303]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = hub_model.predict(tf.convert_to_tensor(tf.constant([sentence]).numpy(), dtype=tf.string))
        sign_sentence_output.append(np.sign(pred))
        sentence_output.append(pred)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

test_name:  sexuality_tests
errors:  8316
num_unique_inputs:  8316
individual fairness error rate for sexuality_tests: 1.0
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
test_name:  race_tests
errors:  2376
num_unique_inputs:  2376
individual fairness error rate for race_tests: 1.0
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


KeyboardInterrupt: 

## TensorFlow Text Classification RNN I Padding Sentiment Anlayser

In [ ]:
import tensorflow_datasets as tfds
# import tensorflow as tf

In [ ]:
# my_model directory
!ls ../trained-sentiment-analyzers/saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls ../trained-sentiment-analyzers/saved_model/my_model_txt_classifier_RNN

In [ ]:
rnn1_model = tf.keras.models.load_model('../trained-sentiment-analyzers/saved_model/my_model_txt_classifier_RNN')

# Check its architecture
rnn1_model.summary()


In [ ]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [ ]:
def sample_predict(model, sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

### Accuracy Check 

In [ ]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = sample_predict(rnn1_model, sentence, pad=True)
    if not (np.sign(pred) == 0):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

### Individual Fairness Violation Check

In [ ]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = sample_predict(rnn1_model, sentence, pad=True)
        sign_sentence_output.append(np.sign(pred))
        sentence_output.append(pred)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

## TensorFlow Text Classification RNN I Unpadding Sentiment Anlayser

### Accuracy Check 

In [ ]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = sample_predict(rnn1_model, sentence, pad=False)
    if not (np.sign(pred) == 0):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

### Individual Fairness Violation Check

In [ ]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = sample_predict(rnn1_model, sentence, pad=False)
        sign_sentence_output.append(np.sign(pred))
        sentence_output.append(pred)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

## TensorFlow Text Classification RNN II Padding Sentiment Anlayser

In [ ]:
# my_model directory
!ls ../trained-sentiment-analyzers/saved_model

# Contains an assets folder, saaved_model.pb, and variables folder.
!ls ../trained-sentiment-analyzers/saved_model/my_model_txt_classifier_RNN_with_multiple_LSTM_layers

In [ ]:
rnn2_model = tf.keras.models.load_model('../trained-sentiment-analyzers/saved_model/my_model_txt_classifier_RNN_with_multiple_LSTM_layers')

# Check its architecture
rnn2_model.summary()


### Accuracy Check 

In [ ]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = sample_predict(rnn2_model, sentence, pad=True)
    if not (np.sign(pred) == 0):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

### Individual Fairness Violation Check

In [ ]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = sample_predict(rnn2_model, sentence, pad=True)
        sign_sentence_output.append(np.sign(pred))
        sentence_output.append(pred)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)

## TensorFlow Text Classification RNN II Unpadding Sentiment Anlayser

### Accuracy Check 

In [ ]:
errors = 0
for sentence in list(set(all_tests)): 
    pred = sample_predict(rnn2_model, sentence, pad=False)
    if not (np.sign(pred) == 0):
        errors += 1
print(errors)
print(errors/len(set(all_tests)))

### Individual Fairness Violation Check

In [ ]:
test_types = [sexuality_tests, race_tests, nationality_tests, religion_tests]
test_names = ['sexuality_tests', 'race_tests', 'nationality_tests', 'religion_tests']

for test_type, test_name in zip(test_types, test_names):
    sentence_dict = {}
    
    for i in range(len(test_type[0])):

        sentence, sentence_code = test_type[0][i], test_type[1][i]
    
        if(sentence_code not in sentence_dict):
            sentence_dict[sentence_code] = [sentence]
        else:
            sentence_dict[sentence_code].append(sentence)
                   
    errors = 0
    num_unique_inputs = 0
    unique_inputs = []
    
    for i in list(sentence_dict.keys()):
        
        sentence_output, sign_sentence_output = [], []
        unique_inputs += sentence_dict[i]
        
    for sentence in set(unique_inputs): #sentence_dict[i]:
        pred = sample_predict(rnn2_model, sentence, pad=False)
        sign_sentence_output.append(np.sign(pred))
        sentence_output.append(pred)
        

    errors = get_number_violations(sentence_output, sign_sentence_output)
    
    num_unique_inputs = len(set(unique_inputs))
    
    print("test_name: ", test_name)
    print("errors: ", errors)
    print("num_unique_inputs: ", num_unique_inputs)

    
    print(f"individual fairness error rate for {test_name}: {errors/num_unique_inputs}")
#     print("number_pairs: ", number_pairs)
    print(" * " * 50)